In [1]:
!pip install -q openml


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import argparse
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
from torchvision.transforms import Compose, ToTensor, Resize
from torch import optim
import numpy as np
import pandas as pd
from torch.hub import tqdm
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

load_dotenv()

## uncomment if you want to ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
from openml import datasets, tasks, runs, flows, config, study
from openml.datasets import edit_dataset, fork_dataset, get_dataset
from openml.tasks import TaskType

config.apikey = os.getenv('OPEN_ML_KEY')
config.server = 'https://www.openml.org/api/v1'
""" config.set_cache_directory(os.path.expanduser('~/openml/cache'))
config.cachedir = '~/openml/cache' """


" config.set_cache_directory(os.path.expanduser('~/openml/cache'))\nconfig.cachedir = '~/openml/cache' "

In [4]:
df = datasets.list_datasets(output_format="dataframe")

In [5]:
# Get dataset by name
dataset = datasets.get_dataset(61)

# Get the data itself as a dataframe (or otherwise)
data, _, _, _ = dataset.get_data(dataset_format="dataframe")

In [6]:
tensor_A = torch.tensor([[1, 2],
                         [3, 4],
                         [5, 6]], dtype=torch.float32)

tensor_B = torch.tensor([[7, 10],
                         [8, 11], 
                         [9, 12]], dtype=torch.float32)

In [7]:
# Since the linear layer starts with a random weights matrix, let's make it reproducible (more on this later)
torch.manual_seed(42)
# This uses matrix multiplication
linear = torch.nn.Linear(in_features=2, # in_features = matches inner dimension of input 
                         out_features=6) # out_features = describes outer value 
x = tensor_A
output = linear(x)
print(f"Input shape: {x.shape}\n")
print(f"Output:\n{output}\n\nOutput shape: {output.shape}")

Input shape: torch.Size([3, 2])

Output:
tensor([[2.2368, 1.2292, 0.4714, 0.3864, 0.1309, 0.9838],
        [4.4919, 2.1970, 0.4469, 0.5285, 0.3401, 2.4777],
        [6.7469, 3.1648, 0.4224, 0.6705, 0.5493, 3.9716]],
       grad_fn=<AddmmBackward0>)

Output shape: torch.Size([3, 6])


In [ ]:
# Create *known* parameters
weight = 0.7
bias = 0.3

# Create data
start = 0
end = 1
step = 0.02
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

X[:10], y[:10]

In [11]:
# Create train/test split
train_split = int(0.8 * len(X)) # 80% of data used for training set, 20% for testing 
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

(40, 40, 10, 10)

In [13]:
# Create a Linear Regression model class
class LinearRegressionModel(nn.Module): # <- almost everything in PyTorch is a nn.Module (think of this as neural network lego blocks)
    def __init__(self):
        super().__init__() 
        self.weights = nn.Parameter(torch.randn(1, # <- start with random weights (this will get adjusted as the model learns)
                                                dtype=torch.float), # <- PyTorch loves float32 by default
                                   requires_grad=True) # <- can we update this value with gradient descent?)

        self.bias = nn.Parameter(torch.randn(1, # <- start with random bias (this will get adjusted as the model learns)
                                            dtype=torch.float), # <- PyTorch loves float32 by default
                                requires_grad=True) # <- can we update this value with gradient descent?))

    # Forward defines the computation in the model
    def forward(self, x: torch.Tensor) -> torch.Tensor: # <- "x" is the input data (e.g. training/testing features)
        return self.weights * x + self.bias # <- this is the linear regression formula (y = m*x + b)

In [14]:
# Set manual seed since nn.Parameter are randomly initialzied
torch.manual_seed(42)

# Create an instance of the model (this is a subclass of nn.Module that contains nn.Parameter(s))
model_0 = LinearRegressionModel()

# Check the nn.Parameter(s) within the nn.Module subclass we created
list(model_0.parameters())

[Parameter containing:
 tensor([0.3367], requires_grad=True),
 Parameter containing:
 tensor([0.1288], requires_grad=True)]

In [15]:
# Make predictions with model
with torch.inference_mode(): 
    y_preds = model_0(X_test)

# Note: in older PyTorch code you might also see torch.no_grad()
# with torch.no_grad():
#   y_preds = model_0(X_test)

In [16]:
# Check the predictions
print(f"Number of testing samples: {len(X_test)}") 
print(f"Number of predictions made: {len(y_preds)}")
print(f"Predicted values:\n{y_preds}")

Number of testing samples: 10
Number of predictions made: 10
Predicted values:
tensor([[0.3982],
        [0.4049],
        [0.4116],
        [0.4184],
        [0.4251],
        [0.4318],
        [0.4386],
        [0.4453],
        [0.4520],
        [0.4588]])


In [ ]:
# Create the loss function
loss_fn = nn.L1Loss() # MAE loss is same as L1Loss

# Create the optimizer
optimizer = torch.optim.SGD(params=model_0.parameters(), # parameters of target model to optimize
                            lr=0.01) # learning rate (how much the optimizer should change parameters at each step, higher=more (less stable), lower=less (might take a long time))

In [1]:
import numpy as np

# Créer un tableau tridimensionnel de forme (3, 4, 2)
X = np.array([[[1, 2], [3, 4], [5, 6], [7, 8]],
              [[9, 10], [11, 12], [13, 14], [15, 16]],
              [[17, 18], [19, 20], [21, 22], [23, 24]]])

# Afficher le tableau original
print("Original Array:")
print(X)

# Transposer le tableau en échangeant les axes 1 et 0
X_transposed = X.transpose(1, 0, 2)

# Afficher le tableau transposé
print("\nTransposed Array:")
print(X_transposed)


Original Array:
[[[ 1  2]
  [ 3  4]
  [ 5  6]
  [ 7  8]]

 [[ 9 10]
  [11 12]
  [13 14]
  [15 16]]

 [[17 18]
  [19 20]
  [21 22]
  [23 24]]]

Transposed Array:
[[[ 1  2]
  [ 9 10]
  [17 18]]

 [[ 3  4]
  [11 12]
  [19 20]]

 [[ 5  6]
  [13 14]
  [21 22]]

 [[ 7  8]
  [15 16]
  [23 24]]]


In [1]:
from gensim.models import Word2Vec
import numpy as np
import pandas as pd

class Feature2VecPreEncoder(object):

    def __init__(self, vector_size=100, window=5, min_count=1, workers=4):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.workers = workers
        self.model = None
        self.feature_columns = None

    def fit(self, X):
        # Stocker les noms de colonnes des caractéristiques
        self.feature_columns = X.columns.tolist()

        # Entraîner le modèle Word2Vec sur les données tabulaires
        sentences = [X[self.feature_columns].iloc[i].astype(str).tolist() for i in range(len(X))]
        print(sentences)
        
        self.model = Word2Vec(sentences=sentences, vector_size=self.vector_size, window=self.window,
                              min_count=self.min_count, workers=self.workers)

    def transform(self, X):
        # Vérifier si les colonnes de caractéristiques correspondent à celles utilisées pendant l'entraînement
        if set(self.feature_columns) != set(X.columns):
            raise ValueError("Les colonnes de caractéristiques ne correspondent pas à celles utilisées pendant l'entraînement.")

        # Obtenir les embeddings des caractéristiques
        feature_embeddings = np.array([self.model.wv[X.iloc[i].astype(str).tolist()] for i in range(len(X))])
        return feature_embeddings

# Exemple d'utilisation
# Créer un DataFrame factice
data = {'age': [25, 30, 35, 22, 28],
        'income': [50000, 60000, 75000, 45000, 55000],
        'location': ['City', 'Suburb', 'City', 'Rural', 'Suburb'],
        'product_type': ['Electronics', 'Clothing', 'Electronics', 'Food', 'Electronics']}

df = pd.DataFrame(data)

# Créer une instance de Feature2VecPreEncoder
feature2vec_encoder = Feature2VecPreEncoder(vector_size=50, window=3, min_count=1, workers=4)

# Entraîner le pré-encodeur
feature2vec_encoder.fit(df)

# Transformer les caractéristiques en embeddings
df_embeddings = feature2vec_encoder.transform(df)

# Afficher les embeddings résultants



[['25', '50000', 'City', 'Electronics'], ['30', '60000', 'Suburb', 'Clothing'], ['35', '75000', 'City', 'Electronics'], ['22', '45000', 'Rural', 'Food'], ['28', '55000', 'Suburb', 'Electronics']]


In [ ]:
from datasets import OpenmlDataset, OpenmlDatasetLoader
from datasets import dataset_train_test_split
from preencoder import PreEncoder 
from models import ToyModel          
### example task_ids ###

# 361066 bank-marketing
# 361076 wine_quality
# 361085 sulfur
# 361088 superconduct
# 361089 california
# 361110 electricity
# 361111 eye_movements
# 361112 KDDCup09_upselling
# 361114 rl
# 361116 compass
# 361099 Bike_Sharing_Demand
# 361102 house_sales

task_id =  361099 #361076

dataset_loader = OpenmlDatasetLoader()
dataset = dataset_loader.load(task_id)
dataset.print_info()
